In [8]:
!dvc get https://github.com/iterative/dataset-registry tutorials/versioning/data.zip

In [10]:
!dvc add data


To track the changes with git, run:

	git add data.dvc

To enable auto staging, run:

	dvc config core.autostage true


In [11]:
!git add data.dvc .gitignore

In [1]:
import numpy as np
import sys
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import applications
from tensorflow.keras.callbacks import CSVLogger
from tqdm.keras import TqdmCallback
from tensorflow import keras 
from PIL import Image

import mlflow
from pathlib import Path

In [2]:
experiment_id = mlflow.create_experiment(
    "cat-dog VGG classifier", 
    artifact_location=Path.cwd().joinpath("./mlruns/mlruns.db").as_uri(),
    tags={"version":"v1", "priority":"P1"}
)

In [3]:
mlflow.set_tracking_uri("sqlite:///mlruns/mlruns.db")

In [4]:
experiment = mlflow.set_experiment(experiment_name="cat-dog VGG classifier")

2022/09/16 00:05:33 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2022/09/16 00:05:34 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [2]:
path = os.getcwd()

In [3]:
img_width, img_height = 150, 150

In [4]:
top_model_weights_path = 'model.h5'
train_data_dir = os.path.join('data', 'train')
validation_data_dir = os.path.join('data', 'validation')
cats_train_path = os.path.join(path, train_data_dir, 'cats')
nb_train_samples = 2 * len([name for name in os.listdir(cats_train_path)
                            if os.path.isfile(
                                os.path.join(cats_train_path, name))])


In [5]:
batch_size = 1

In [6]:
datagen = ImageDataGenerator(rescale=1. / 255)

# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

val_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

Found 1000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [7]:
model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))
x = model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)


In [8]:
optimizer = 'sgd'
loss_fn = 'binary_crossentropy'

In [9]:
model.compile(optimizer=optimizer,
              loss=loss_fn, metrics=[keras.metrics.Accuracy(), keras.metrics.Precision(), keras.metrics.Recall()])

In [10]:
epochs = 300

In [11]:
# with mlflow.start_run(run_name=optimizer,experiment_id=experiment_id, description="A model to classify between cat and dog example") as run:

mlflow.keras.autolog(registered_model_name="VGG cat-dog classifier")
model.fit(
    train_generator,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=val_generator,
    verbose=0,
    callbacks=[TqdmCallback(), CSVLogger("metrics.csv")])

  0%|          | 0/300 [00:00<?, ?epoch/s]

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  assertion failed: [predictions must be <= 1] [Condition x <= y did not hold element-wise:] [x (model/dense/BiasAdd:0) = ] [[1.37736058]] [y (Cast_5/x:0) = ] [1]
	 [[{{node assert_less_equal/Assert/AssertGuard/else/_11/assert_less_equal/Assert/AssertGuard/Assert}}]]
	 [[assert_less_equal_1/Assert/AssertGuard/pivot_f/_41/_81]]
  (1) Invalid argument:  assertion failed: [predictions must be <= 1] [Condition x <= y did not hold element-wise:] [x (model/dense/BiasAdd:0) = ] [[1.37736058]] [y (Cast_5/x:0) = ] [1]
	 [[{{node assert_less_equal/Assert/AssertGuard/else/_11/assert_less_equal/Assert/AssertGuard/Assert}}]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_1849]

Function call stack:
train_function -> train_function


In [45]:
model.save_weights(top_model_weights_path)

In [ ]:
!dvc add model.h5 metrics.csv

## Serving a registered model

In [ ]:
import mlflow.pyfunc

In [ ]:
model_name = "VGG cat-dog classifier"
stage = 'Staging'

In [ ]:
model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{stage}"
)

In [20]:
test_img = next(val_generator)

In [34]:
img_name = "dogs\\dog.6.jpg"

In [35]:
test_img = np.array(Image.open(os.path.join(train_data_dir, img_name)))
test_img = np.expand_dims(test_img, axis=0)

In [37]:
test_img.shape

(1, 488, 499, 3)

In [38]:
model.predict(test_img)

WARNI [tensorflow] Model was constructed with shape (None, 4, 4, 512) for input KerasTensor(type_spec=TensorSpec(shape=(None, 4, 4, 512), dtype=tf.float32, name='flatten_input'), name='flatten_input', description="created by layer 'flatten_input'"), but it was called on an input with incompatible shape (None, 488, 499, 3).


ValueError: in user code:

    c:\Users\Mahad\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    c:\Users\Mahad\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\Users\Mahad\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\Users\Mahad\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\Users\Mahad\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_re